In [5]:
import os
import pandas as pd
import requests
import json
import time
import dateutil
import datetime
from dateutil.relativedelta import relativedelta
import urllib3
from dotenv import load_dotenv
import logging
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
logging.basicConfig(filename="logs/scraping.log",level=logging.DEBUG)

In [2]:
load_dotenv('.env')

True

In [3]:
API_KEY = os.environ.get('NEW_YORK_TIMES_API_KEY')


In [6]:
def send_request(date):
    base_url = 'https://api.nytimes.com/svc/archive/v1/'
    url = base_url+'/'+date[0]+'/'+date[1]+'.json?api-key='+API_KEY
    try:
        respose = requests.get(url,verify=False).json()
    except Exception as e:
        logging.warning(f'failed GET request:::{e}')
        return None
    time.sleep(6)
    return respose

In [ ]:
def is_valid(article,date):
    is_in_range = date > start and date < end
    has_headline = type(article['headline'])

In [21]:
def nytimes_query(
    
    api_key, query,
    glocations = None, headline = None, news_desk = None,
    organizations = None, persons = None, byline = None,
    subject = None, news_type = None, type_of_material = None,
    begin_date = None, end_date = None, n_page = 0
    ):
    # Set the base url for the query
    base_url = f'https://api.nytimes.com/svc/search/v2/articlesearch.json?q={query}'

        # Empty dictionary for filters
    filter_queries = {}
    # empty dictionary for dates
    dates = {}
    # Populate the filter dictionary
    if glocations:
        filter_queries.update({'glocations': glocations})
        print(filter_queries)
    if headline:
        filter_queries.update({'headline': headline})
        print(filter_queries)
    if news_desk:
        filter_queries.update({'news_desk': news_desk})
        print(filter_queries)
    if organizations:
        filter_queries.update({'organizations': organizations})
        print(filter_queries)
    if persons:
        filter_queries.update({'persons': persons})
        print(filter_queries)
    if subject:
        filter_queries.update({'subject': subject})
        print(filter_queries)
    if byline:
        filter_queries.update({'byline': byline})
        print(filter_queries)
    if news_type:
        filter_queries.update({'news_type': news_type})
        print(filter_queries)
    if type_of_material:
        filter_queries.update({'type_of_material': type_of_material})
        print(filter_queries)
    # Populate the date dictionary
    if begin_date:
        dates.update({'begin_date': begin_date})
        print(dates)
    if end_date:
        dates.update({'end_date':end_date})
        print(dates)
    # If 1 filter is present, and/or date params, add to URL and execute query
    if len(filter_queries) == 1:
        base_url += f'&fq={list(filter_queries.keys())[0]}:("{list(filter_queries.values())[0]}")'
        print(base_url)
        if len(dates) == 1:
            base_url += f'&{list(dates.keys())[0]}={list(dates.values())[0]}'
            print(base_url)
        elif len(dates) == 2:
            base_url += '&'
            for i in dates.keys():
                base_url += f'{i}={dates[i]}&'
            print(base_url)
    # If 2 or more filters are present, concatenate with AND, add dates if present and execute
    elif  len(filter_queries) > 1:
        base_url += '&fq='
        for i in filter_queries.keys():
            base_url += f'{i}:({filter_queries[i]}) AND '
        # remove the last 'AND ' at the end of the loop
        base_url = base_url[:-5]
        print(base_url)

        if len(dates) == 1:
            base_url += f'&{list(filter_queries.keys())[0]}={list(filter_queries.values())[0]}&'
            print(base_url)

        elif len(dates) == 2:
            base_url += '&'
            print(base_url)

            for i in dates.keys():
                base_url += f'{i}={dates[i]}&'
                print(base_url)

    # concatenate page number and api key and make the request.
    # Returns a truncated JSON indexed past the metadata
    # If you want the full json, simply remove ['response']['docs']
    # from the return line
    base_url += f'&page={n_page}'
    base_url += f'&api-key={api_key}'
    print(f'Final query : {base_url}')
    r = requests.get(base_url)
    json_data = r.json()
    return r.json()['response']['docs']


In [25]:
query = nytimes_query(API_KEY,query='Meta',news_desk='Business',begin_date='20220101',end_date='20220114')
query

{'news_desk': 'Business'}
{'begin_date': '20220101'}
{'begin_date': '20220101', 'end_date': '20220114'}
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=Meta&fq=news_desk:("Business")
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=Meta&fq=news_desk:("Business")&begin_date=20220101&end_date=20220114&
Final query : https://api.nytimes.com/svc/search/v2/articlesearch.json?q=Meta&fq=news_desk:("Business")&begin_date=20220101&end_date=20220114&&page=0&api-key=YPrMAd93hm7pmTqscIQVlnGZiv1EwSOI


[{'abstract': 'A lawsuit argued that individuals who sought to harm law enforcement officers were able to connect with one another on Facebook.',
  'web_url': 'https://www.nytimes.com/2022/01/06/technology/meta-facebook-lawsuit-security-guard.html',
  'snippet': 'A lawsuit argued that individuals who sought to harm law enforcement officers were able to connect with one another on Facebook.',
  'lead_paragraph': 'Facebook’s parent company, Meta Platforms, has been sued over the 2020 killing of a federal security guard, a move that aims to challenge a federal statute that shields websites and social media platforms from liability for what users post.',
  'source': 'The New York Times',
  'multimedia': [],
  'headline': {'main': 'Lawsuit says Meta shares blame in the killing of a federal guard.',
   'kicker': None,
   'content_kicker': None,
   'print_headline': None,
   'name': None,
   'seo': None,
   'sub': None},
  'keywords': [{'name': 'subject',
    'value': 'Suits and Litigation (C

In [26]:
tmp = pd.DataFrame(query)
tmp

,abstract,web_url,snippet,lead_paragraph,source,multimedia,headline,keywords,pub_date,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri,print_section,print_page,subsection_name
0,A lawsuit argued that individuals who sought t...,https://www.nytimes.com/2022/01/06/technology/...,A lawsuit argued that individuals who sought t...,"Facebook’s parent company, Meta Platforms, has...",The New York Times,[],{'main': 'Lawsuit says Meta shares blame in th...,"[{'name': 'subject', 'value': 'Suits and Litig...",2022-01-06T22:48:49+0000,article,Business,Technology,"{'original': 'By Ryan Mac', 'person': [{'first...",News,nyt://article/d24018a0-a4e3-52af-a406-becdfdab...,380,nyt://article/d24018a0-a4e3-52af-a406-becdfdab...,NaN,NaN,NaN
1,The states’ central claim is that Facebook acq...,https://www.nytimes.com/2022/01/14/technology/...,The states’ central claim is that Facebook acq...,WASHINGTON — Nearly four dozen states on Frida...,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'States appeal a judge’s decision to ...,"[{'name': 'subject', 'value': 'Antitrust Laws ...",2022-01-14T23:54:43+0000,article,Business,Technology,"{'original': 'By Cecilia Kang', 'person': [{'f...",News,nyt://article/2d6a2236-d14e-5bca-acc7-e2936a08...,388,nyt://article/2d6a2236-d14e-5bca-acc7-e2936a08...,NaN,NaN,NaN
2,The states argue that Google reached a deal wi...,https://www.nytimes.com/2022/01/14/technology/...,The states argue that Google reached a deal wi...,Google’s chief executive approved an agreement...,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'Google’s chief executive signed off ...,"[{'name': 'subject', 'value': 'Antitrust Laws ...",2022-01-14T17:43:48+0000,article,Business,Technology,"{'original': 'By David McCabe', 'person': [{'f...",News,nyt://article/c8e60dfb-c4ce-5407-a187-0519034d...,433,nyt://article/c8e60dfb-c4ce-5407-a187-0519034d...,B,5,NaN
3,Need a Covid rapid test? One solution is to wo...,https://www.nytimes.com/2022/01/12/business/de...,Need a Covid rapid test? One solution is to wo...,As the Omicron variant of the coronavirus surg...,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","{'main': 'The Newest Corporate Perk', 'kicker'...","[{'name': 'subject', 'value': 'Coronavirus Ret...",2022-01-12T12:33:52+0000,article,Business,Business Day,"{'original': 'By Andrew Ross Sorkin, Jason Kar...",News,nyt://article/bf7a9846-6fcd-51ba-bc1c-50c79415...,1637,nyt://article/bf7a9846-6fcd-51ba-bc1c-50c79415...,NaN,NaN,DealBook
4,The government can proceed with its claims tha...,https://www.nytimes.com/2022/01/11/technology/...,The government can proceed with its claims tha...,A federal judge on Tuesday allowed the Federal...,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'A Facebook antitrust suit can move f...,"[{'name': 'subject', 'value': 'Antitrust Laws ...",2022-01-11T20:54:03+0000,article,Business,Technology,"{'original': 'By Cecilia Kang', 'person': [{'f...",News,nyt://article/5d0bb7af-de7e-53aa-ab6e-0ad76330...,955,nyt://article/5d0bb7af-de7e-53aa-ab6e-0ad76330...,B,1,NaN
5,"The blockchain start-up, one of the most talke...",https://www.nytimes.com/2022/01/04/business/op...,"The blockchain start-up, one of the most talke...","OpenSea, one of the most talked-about blockcha...",The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'OpenSea valued at $13.3 billion in n...,"[{'name': 'organizations', 'value': 'OpenSea (...",2022-01-05T01:09:28+0000,article,Business,Business Day,"{'original': 'By Mike Isaac', 'person': [{'fir...",News,nyt://article/46c6977b-8f33-54ba-b6c0-828e521d...,543,nyt://article/46c6977b-8f33-54ba-b6c0-828e521d...,NaN,NaN,NaN
6,"The internet giant, planning on employees retu...",https://www.nytimes.com/2022/01/11/business/go...,"The internet giant, planning on employees retu...",Google said Friday that it would spend $1 bill...,The New York Times